# Topic 5: Human-in-the-Loop

Add human approval and interaction to your workflows using interrupts and breakpoints. Learn how to create approval workflows where humans can review, approve, or modify AI decisions.

## Learning Objectives

- Implement interrupts for human review
- Create approval workflows
- Resume execution after human input
- Build interactive agent systems

In [ ]:
# Setup
import os
import getpass
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
import uuid

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(model="claude-sonnet-4-20250514")
print("✓ Setup complete!")

## Understanding Human-in-the-Loop

Human-in-the-loop patterns allow you to:
- Pause execution for human review
- Request approval before critical actions
- Allow humans to modify AI decisions
- Resume execution after human input

This is crucial for:
- High-stakes decisions
- Compliance requirements
- Quality control
- Learning and oversight

## Example 1: Blog Post Approval Workflow

Let's build a content creation system that requires human approval before publishing.

In [ ]:
# Define state
class BlogPostState(TypedDict):
    topic: str
    draft: str
    approved: bool
    feedback: str
    final_post: str

print("✓ BlogPostState defined")

In [ ]:
def write_draft(state: BlogPostState) -> BlogPostState:
    """Write initial blog post draft."""
    print("\n✍️  Writing draft...")
    
    prompt = f"""Write a blog post about: {state['topic']}
    
Make it engaging, informative, and around 300 words.
Include:
- Catchy introduction
- Main points with examples
- Clear conclusion"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    print("✓ Draft complete! Ready for review.")
    return {"draft": response.content}

def revise_draft(state: BlogPostState) -> BlogPostState:
    """Revise draft based on human feedback."""
    print("\n🔄 Revising based on feedback...")
    
    prompt = f"""Revise this blog post based on the feedback:

DRAFT:
{state['draft']}

FEEDBACK:
{state['feedback']}

Provide the revised version."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    # Reset approval status for re-review
    return {
        "draft": response.content,
        "approved": False,
        "feedback": ""
    }

def publish_post(state: BlogPostState) -> BlogPostState:
    """Publish the approved post."""
    print("\n🚀 Publishing post...")
    
    return {
        "final_post": state['draft'],
        "approved": True
    }

print("✓ All nodes created")

In [ ]:
def human_review_node(state: BlogPostState) -> BlogPostState:
    """This node triggers a breakpoint for human review."""
    print("\n⏸️  Pausing for human review...")
    print("="*60)
    print("DRAFT FOR REVIEW:")
    print("="*60)
    print(state['draft'])
    print("="*60)
    
    # The interrupt will happen when we add interrupt_before to compile
    # For now, we'll simulate it
    return state

print("✓ Review node created")

In [ ]:
def route_after_review(state: BlogPostState) -> Literal["publish", "revise"]:
    """Route based on approval status."""
    if state.get('approved', False):
        print("✅ Approved! Proceeding to publish...")
        return "publish"
    else:
        print("📝 Needs revision...")
        return "revise"

print("✓ Router created")

## Building the Graph with Interrupts

Key point: We use `interrupt_before` to pause execution at specific nodes.

In [ ]:
# Create graph
graph_builder = StateGraph(BlogPostState)

# Add nodes
graph_builder.add_node("write_draft", write_draft)
graph_builder.add_node("human_review", human_review_node)
graph_builder.add_node("revise", revise_draft)
graph_builder.add_node("publish", publish_post)

# Add edges
graph_builder.add_edge(START, "write_draft")
graph_builder.add_edge("write_draft", "human_review")

# After review, route based on approval
graph_builder.add_conditional_edges(
    "human_review",
    route_after_review,
    {
        "publish": "publish",
        "revise": "revise"
    }
)

# After revision, go back to review
graph_builder.add_edge("revise", "human_review")
graph_builder.add_edge("publish", END)

# CRITICAL: Add checkpointer and interrupt_before
memory = MemorySaver()
blog_graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["human_review"]  # Pause before review
)

print("✓ Blog approval graph compiled with interrupts!")

## Visualize the Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(blog_graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph structure:")
    print("START -> write_draft -> [INTERRUPT] human_review -> {publish|revise -> human_review} -> END")

## Step 1: Start the Workflow

The graph will pause at the interrupt point.

In [ ]:
# Create a thread ID for this execution
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

print(f"Thread ID: {thread_id}")
print("\nStarting workflow...")
print("="*60)

# Start the workflow
initial_state = {
    "topic": "The Benefits of LangGraph for Building AI Agents",
    "draft": "",
    "approved": False,
    "feedback": "",
    "final_post": ""
}

# This will execute until the interrupt
for event in blog_graph.stream(initial_state, config):
    for node, state in event.items():
        print(f"Node '{node}' executed")

print("\n⏸️  Workflow paused at interrupt point.")

## Step 2: Review the Current State

In [ ]:
# Get current state
current_state = blog_graph.get_state(config)

print("Current State:")
print("="*60)
print(f"Next Node: {current_state.next}")
print(f"\nDraft:\n{current_state.values['draft']}")
print("="*60)

## Step 3: Approve or Request Revision

Now we simulate human decision-making by updating the state.

In [ ]:
# Option 1: APPROVE
print("Human Decision: APPROVE")
print("\nUpdating state to approve...")

# Update the state with approval
blog_graph.update_state(
    config,
    {"approved": True}
)

print("✓ State updated. Resuming workflow...")
print("="*60)

# Resume execution
for event in blog_graph.stream(None, config):
    for node, state in event.items():
        print(f"Node '{node}' executed")

print("\n✓ Workflow complete!")

## Example with Revision

Let's try a workflow where we request revisions:

In [ ]:
# Start new workflow
thread_id_2 = str(uuid.uuid4())
config_2 = {"configurable": {"thread_id": thread_id_2}}

print(f"New Thread ID: {thread_id_2}")
print("\nStarting new workflow...")
print("="*60)

initial_state_2 = {
    "topic": "Why Developers Love Python",
    "draft": "",
    "approved": False,
    "feedback": "",
    "final_post": ""
}

# Run until interrupt
for event in blog_graph.stream(initial_state_2, config_2):
    pass

print("⏸️  Paused for review")

In [ ]:
# Review and request changes
print("Human Decision: REQUEST REVISION")
print("\nProviding feedback...")

# Update state with feedback (not approved)
blog_graph.update_state(
    config_2,
    {
        "approved": False,
        "feedback": "Please add more specific code examples and make it more technical."
    }
)

print("✓ Feedback provided. Resuming for revision...")
print("="*60)

# Resume - will go to revise node
for event in blog_graph.stream(None, config_2):
    for node, state in event.items():
        print(f"Node '{node}' executed")

print("\n⏸️  Paused again for re-review after revision")

In [ ]:
# Check revised draft
current_state_2 = blog_graph.get_state(config_2)

print("Revised Draft:")
print("="*60)
print(current_state_2.values['draft'])
print("="*60)

In [ ]:
# Now approve
print("\nHuman Decision: APPROVE REVISION")

blog_graph.update_state(config_2, {"approved": True})

# Resume and complete
for event in blog_graph.stream(None, config_2):
    for node, state in event.items():
        print(f"Node '{node}' executed")

print("\n✓ Workflow complete with revisions!")

## Example 2: Expense Approval System

Let's build a practical expense approval workflow:

In [ ]:
class ExpenseState(TypedDict):
    amount: float
    category: str
    description: str
    approved: bool
    rejection_reason: str
    auto_approved: bool

def check_auto_approve(state: ExpenseState) -> ExpenseState:
    """Check if expense can be auto-approved."""
    print(f"\n💰 Processing expense: ${state['amount']}")
    
    # Auto-approve small expenses
    if state['amount'] < 100:
        print("✓ Auto-approved (under $100)")
        return {
            "approved": True,
            "auto_approved": True
        }
    else:
        print("⚠️  Requires manual approval")
        return {"auto_approved": False}

def process_approval(state: ExpenseState) -> ExpenseState:
    """Process the expense approval."""
    if state['approved']:
        print("\n✅ EXPENSE APPROVED")
        print(f"Amount: ${state['amount']}")
        print(f"Category: {state['category']}")
    else:
        print("\n❌ EXPENSE REJECTED")
        print(f"Reason: {state.get('rejection_reason', 'Not specified')}")
    
    return state

def route_approval(state: ExpenseState) -> Literal["process", "manual_review"]:
    """Route based on auto-approval."""
    if state.get('auto_approved', False):
        return "process"
    return "manual_review"

def manual_review_node(state: ExpenseState) -> ExpenseState:
    """Pause for manual review."""
    print("\n👤 MANUAL REVIEW REQUIRED")
    print("="*60)
    print(f"Amount: ${state['amount']}")
    print(f"Category: {state['category']}")
    print(f"Description: {state['description']}")
    print("="*60)
    return state

print("✓ Expense nodes created")

In [ ]:
# Build expense graph
expense_builder = StateGraph(ExpenseState)

expense_builder.add_node("check_auto", check_auto_approve)
expense_builder.add_node("manual_review", manual_review_node)
expense_builder.add_node("process", process_approval)

expense_builder.add_edge(START, "check_auto")

expense_builder.add_conditional_edges(
    "check_auto",
    route_approval,
    {
        "process": "process",
        "manual_review": "manual_review"
    }
)

expense_builder.add_edge("manual_review", "process")
expense_builder.add_edge("process", END)

# Compile with interrupt
expense_memory = MemorySaver()
expense_graph = expense_builder.compile(
    checkpointer=expense_memory,
    interrupt_before=["manual_review"]
)

print("✓ Expense approval graph compiled!")

## Test Auto-Approval (Small Expense)

In [ ]:
# Small expense - auto-approved
thread_3 = str(uuid.uuid4())
config_3 = {"configurable": {"thread_id": thread_3}}

print("Test 1: Small Expense (Auto-Approval)")
print("="*60)

result = expense_graph.invoke(
    {
        "amount": 45.50,
        "category": "Office Supplies",
        "description": "Notebooks and pens",
        "approved": False,
        "rejection_reason": "",
        "auto_approved": False
    },
    config_3
)

print("\n✓ Completed without manual review!")

## Test Manual Approval (Large Expense)

In [ ]:
# Large expense - needs approval
thread_4 = str(uuid.uuid4())
config_4 = {"configurable": {"thread_id": thread_4}}

print("\nTest 2: Large Expense (Manual Approval Required)")
print("="*60)

for event in expense_graph.stream(
    {
        "amount": 1250.00,
        "category": "Equipment",
        "description": "New laptop for development",
        "approved": False,
        "rejection_reason": "",
        "auto_approved": False
    },
    config_4
):
    pass

print("\n⏸️  Awaiting manager approval...")

In [ ]:
# Manager approves
print("Manager Decision: APPROVED")

expense_graph.update_state(config_4, {"approved": True})

# Resume
for event in expense_graph.stream(None, config_4):
    pass

print("\n✓ Expense processed!")

## Key Concepts

### 1. Checkpointer
Required for interrupts - saves state so execution can resume

### 2. Thread ID
Identifies a specific execution instance

### 3. Interrupt Points
Use `interrupt_before` or `interrupt_after` to pause execution

### 4. State Updates
Use `update_state()` to modify state during interrupts

### 5. Resumption
Call `stream(None, config)` to resume from interrupt

## Exercise: Build a Code Deployment Approval System

Create a deployment workflow that:
1. Runs tests
2. Pauses for QA approval
3. If approved, pauses for production approval
4. Deploys to production
5. If rejected at any stage, rolls back

Hint: Use multiple interrupt points for multi-stage approval!

In [ ]:
# Your code here!

class DeploymentState(TypedDict):
    version: str
    tests_passed: bool
    qa_approved: bool
    prod_approved: bool
    deployed: bool
    rollback: bool

# TODO: Create nodes for:
# - run_tests
# - qa_review
# - prod_review
# - deploy
# - rollback

# TODO: Build graph with multiple interrupt points
# TODO: Test different approval scenarios

## Key Takeaways

In this notebook, you learned:

1. ✅ How to add interrupts to workflows using `interrupt_before`
2. ✅ Using checkpointers to save and resume state
3. ✅ Updating state during interrupts with `update_state()`
4. ✅ Building approval workflows with human review
5. ✅ Creating multi-stage approval processes
6. ✅ Combining automatic and manual decision points

## Next Steps

Continue to **Topic 6: Subgraphs** to learn how to create modular, reusable graph components!